In [78]:
%load_ext autoreload
%autoreload 2

import pprint

In [79]:
import config
import re
from dataclasses import dataclass
from typing import List

from llama_index.core import Document, Settings, VectorStoreIndex, ChatPromptTemplate
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core.node_parser import SimpleNodeParser, SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.anthropic import Anthropic

api_keys = config.Config("../api_keys.cfg")

Settings.llm = None

LLM is explicitly disabled. Using MockLLM.


In [80]:
anthropic_api_key = api_keys["ANTHROPIC_API_KEY"]
small_llm = Anthropic(model="claude-3-haiku-20240307", api_key=anthropic_api_key)

In [81]:
# response = small_llm.complete("What is the square root of 44? Manually do this using Newton's method.")
# response_text = response.text
# print(response_text)

In [82]:
# TODO: construct prompt templates using a list of str lines, then later join the lines with "\n" to get the full prompt format

memory_query_formulation_template = ChatPromptTemplate(message_templates=[
    ChatMessage(
        content=(
            "You are a professor of cognitive science specializing in human associational memory and information retrieval processes. "
            "Your task is to formulate psychologically plausible search queries over a person's long-term memory based on their current cognitive state."
        ),
        role=MessageRole.SYSTEM
    ),
    ChatMessage(
        content=(
            "Given a written summary of the contents of a person's current working memory in <working_memory> tags and their current observations in <observations>, "
            "formulate search queries over the person's long-term memory to find relevant memories. The long-term memory is implemented as a vector database of semantic text-embeddings. "
            "Output the queries you decide on in <query_1></query_1>, <query_2></query_2>, ..., <query_n></query_n> tags."
            "\n\n"
            "While formulating the queries:\n"
            "1. Generate a diverse set covering the full range of associations the person would likely make in about 10 seconds.\n"
            "2. Provide between 3 and 10 queries.\n"
            "3. Consider the emotional salience of information in working memory and observations.\n"
            "4. Reflect the person's current goals and motivations in the query formulation.\n"
            "5. Account for common cognitive biases such as the recency effect, availability heuristic, and confirmation bias.\n"
            "6. Include both specific episodic memory queries and broader semantic knowledge queries.\n"
            "7. Consider the temporal context and how it might influence memory retrieval.\n"
            "8. Incorporate metacognitive aspects, such as the person's awareness of their own knowledge gaps."
            "\n\n"
            "Ensure that your queries are psychologically plausible and reflect the complex, associative nature of human memory retrieval."
            "\n\n"
            "Here are the current working memory and observations for reference:\n"
            "<working_memory>\n"
            "{working_memory}\n"
            "</working_memory>\n"
            "\n\n"
            "<observations>\n"
            "{observations}\n"
            "</observations>\n"
        ),
        role=MessageRole.USER,
    ),
])


memory_report_synthesis_template = ChatPromptTemplate(message_templates=[
    ChatMessage(
        content=(
            "You are an expert cognitive scientist specializing in memory consolidation and retrieval processes. "
            "Your task is to create a psychologically plausible memory report based on the given working memory and retrieved associations."
        ),
        role=MessageRole.SYSTEM
    ),
    ChatMessage(
        content=(
            "Recent neuroscience research has shown that memory consolidation involves the hippocampus and neocortex working together to integrate new information with existing knowledge. "
            "This process is influenced by factors such as emotional salience, relevance to current goals, and the strength of neural connections.\n\n"
            "Given the current working memory in <working_memory> tags and retrieved memories in <retrieved_memories> tags, create a memory report in <memory_report> tags. This report should:\n\n"
            "1. Synthesize and compile all interesting, useful, relevant, and important information from the memories.\n"
            "2. Incorporate and refine information from the working memory.\n"
            "3. Prioritize information based on emotional significance and relevance to current goals.\n"
            "4. Reflect the associative nature of human memory by highlighting connections between different pieces of information.\n"
            "5. Include both episodic (event-specific) and semantic (general knowledge) elements.\n"
            "6. Demonstrate the reconstructive nature of memory by filling in gaps with plausible inferences.\n"
            "7. Account for the temporal dynamics of memory retrieval, with more recent or emotionally charged memories potentially being more vivid.\n"
            "8. If appropriate, include metacognitive reflections on the reliability or completeness of recalled information.\n"
            "9. Consider how common cognitive biases (e.g., confirmation bias, hindsight bias) might influence the memory reconstruction process.\n"
            "10. Reflect individual differences in memory processing by allowing for some variability in the synthesis approach.\n\n"
            "Ensure that the memory report is coherent, psychologically plausible, and supports the agent's competence in decision-making and problem-solving. "
            "The report should read as a natural flow of thoughts rather than a structured list."
            "\n\n"
            "Here is the current working memory and retrieved memories for reference:\n"
            "<working_memory>\n"
            "{working_memory}\n"
            "</working_memory>\n"
            "\n\n"
            "<retrieved_memories>\n"
            "{retrieved_memories}\n"
            "</retrieved_memories>\n"
        ),
        role=MessageRole.USER
    ),
])


working_memory_update_template = ChatPromptTemplate(message_templates=[
    ChatMessage(
        content=(
            "You are a cognitive neuroscientist specializing in working memory dynamics and attentional processes. "
            "Your task is to update the agent's working memory in a psychologically plausible manner based on the given memory report and current working memory contents."
        ),
        role=MessageRole.SYSTEM
    ),
    ChatMessage(
        content=(
            "Recent research has shown that working memory is not just a passive storage system but an active process involving the prefrontal cortex and basal ganglia. "
            "It has limited capacity (typically 4-7 items) and is subject to decay and interference.\n\n"
            "Given the memory report in <memory_report> tags and the current working memory in <working_memory> tags, update the working memory and provide the result in <updated_working_memory> tags. In this process:\n\n"
            "1. Maintain a limited capacity of 4-7 main items or chunks of information.\n"
            "2. Prioritize new, emotionally salient, and goal-relevant information from the memory report.\n"
            "3. Integrate new information with existing knowledge in working memory.\n"
            "4. Allow for some decay of older or less relevant information.\n"
            "5. Account for recency and primacy effects in information retention.\n"
            "6. Reflect the influence of attentional focus on working memory contents.\n"
            "7. Include meta-cognitive elements such as current goals and task-relevant strategies.\n"
            "8. Consider how emotional state might influence working memory updating and capacity.\n"
            "9. Incorporate the agent's level of arousal or stress, which can affect working memory function.\n"
            "10. Allow for some variability in working memory capacity and updating efficiency to reflect individual differences.\n"
            "11. Consider how the agent's current cognitive load might affect the updating process.\n"
            "12. Reflect the associative nature of working memory by highlighting connections between items.\n\n"
            "Present the updated working memory as a coherent stream of thoughts rather than a structured list. "
            "Ensure that it is concise, psychologically plausible, and supports effective cognitive processing for the agent."
            "\n\n"
            "Here is the current working memory and memory report for reference:\n"
            "<working_memory>\n"
            "{working_memory}\n"
            "</working_memory>\n"
            "\n\n"
            "<memory_report>\n"
            "{memory_report}\n"
            "</memory_report>\n"
        ),
        role=MessageRole.USER
    ),
])


action_decision_template = ChatPromptTemplate(message_templates=[
    ChatMessage(
        content=(
            "You are an expert in cognitive psychology and decision-making processes. "
            "Your task is to determine the most likely action an agent would take based on their current working memory and available actions, in a psychologically plausible manner."
        ),
        role=MessageRole.SYSTEM
    ),
    ChatMessage(
        content=(
            "Recent research in decision neuroscience has shown that human decision-making involves complex interactions between the prefrontal cortex, basal ganglia, and limbic system. "
            "This process integrates cognitive, emotional, and motivational factors, often using heuristics and being influenced by biases.\n\n"
            "Given the working memory in <working_memory> tags and a description of the available actions in <actions>, choose which action the agent would be most likely to take and specify how it should be executed. "
            "Respond in the format specified in <actions> within <action_decision> tags. In making this decision:\n\n"
            "1. Consider the agent's current goals, emotional state, and motivations as reflected in working memory.\n"
            "2. Evaluate the potential outcomes of each action in relation to the agent's goals.\n"
            "3. Account for the agent's past experiences and learned patterns of behavior.\n"
            "4. Incorporate elements of bounded rationality, recognizing that the agent has limited cognitive resources and may use heuristics.\n"
            "5. Allow for the influence of cognitive biases such as confirmation bias or availability heuristic.\n"
            "6. Consider the role of intuition and gut feelings in quick decision-making.\n"
            "7. Reflect the agent's risk tolerance and time preferences.\n"
            "8. Include any necessary preparation or planning steps as part of the action execution.\n"
            "9. Consider how the agent's current physiological state (e.g., fatigue, hunger) might influence the decision.\n"
            "10. Incorporate metacognitive aspects, such as the agent's confidence in their decision.\n"
            "11. Allow for some variability in decision-making to reflect individual differences and the probabilistic nature of human choice.\n"
            "12. Consider how social factors or perceived social norms might influence the decision.\n"
            "13. Reflect the associative nature of decision-making by considering how the chosen action might relate to or trigger other potential actions.\n\n"
            "Ensure that the chosen action and its execution are psychologically plausible, consistent with the agent's current cognitive state, and likely to be effective in pursuing the agent's goals. "
            "Present the decision process and chosen action as a natural flow of thoughts rather than a structured list."
            "\n\n"
            "Here is the current working memory and available actions for reference:\n"
            "<working_memory>\n"
            "{working_memory}\n"
            "</working_memory>\n"
            "\n\n"
            "<actions>\n"
            "{actions}\n"
            "</actions>\n"
        ),
        role=MessageRole.USER
    ),
])

In [83]:
class LLMResponseGenerator:
    def __init__(
            self,
            input_tags: List[str],
            output_tag_patterns: List[str],
            prompt_template: ChatPromptTemplate,
            llm: Anthropic,
    ):
        self.input_tags = input_tags
        self.output_tag_patterns = output_tag_patterns
        self.prompt_template = prompt_template
        self.llm = llm

    def generate_response(self, **input_strs) -> str:
        if not all(tag in input_strs for tag in self.input_tags):
            missing_tags = [tag for tag in self.input_tags if tag not in input_strs]
            raise ValueError(f"Missing required input tags: {missing_tags}")

        messages = self.prompt_template.format_messages(**input_strs)
        response = self.llm.chat(messages)

        output = response.message.content
        output_tags_content = self._extract_output_tags(output)

        return {**output_tags_content, "response": output}

    def _extract_output_tags(self, output: str) -> dict:
        output_tags_content = {}
        for tag_pattern in self.output_tag_patterns:
            output_tags_content[tag_pattern] = {}
            matching_tags = re.findall(f"<({tag_pattern})>", output, re.DOTALL)
            for tag in matching_tags:
                content = self._extract_tag_content(output, tag)
                output_tags_content[tag_pattern][tag] = content

            # If the only matching tag is the pattern itself, map directly from pattern -> full output
            if len(output_tags_content[tag_pattern]) == 1 and tag_pattern == list(output_tags_content.keys())[0]:
                output_tags_content[tag_pattern] = output_tags_content[tag_pattern][tag_pattern]

        return output_tags_content

    @staticmethod
    def _extract_tag_content(text: str, tag: str) -> str:
        pattern = f"<{tag}>(.*?)</{tag}>"
        match = re.search(pattern, text, re.DOTALL)
        if match:
            return match.group(1).strip()
        return None

test_working_memory = (
    "I am walking through a forest. The trees are tall and the air is fresh. I feel a sense of peace and tranquility.",
)
test_observations = (
    "I notice a squirrel running up a tree. The sunlight filters through the leaves, creating dappled patterns on the ground.",
    "I hear the chirping of birds and the rustling of leaves in the gentle breeze.",
    "I feel the coolness of the air on my skin and the softness of the ground beneath my feet.",
    "I smell the earthy scent of moss and the sweet fragrance of wildflowers.",
)
test_retrieved_memories = (
    "I remember a childhood camping trip with my family. We roasted marshmallows over a campfire and told stories under the stars.",
    "I recall a biology class where we learned about photosynthesis and the importance of trees in the ecosystem.",
    "I think about a recent article I read on the benefits of spending time in nature for mental health and well-being.",
)


In [85]:
query_tag_pattern = r"query_\d+"
query_response_generator = LLMResponseGenerator(
    input_tags=["working_memory", "observations"],
    output_tag_patterns=[query_tag_pattern],
    prompt_template=memory_query_formulation_template,
    llm=small_llm,
)

query_response = query_response_generator.generate_response(
    working_memory=test_working_memory,
    observations=test_observations,
)
print(query_response["response"])

{'query_1': 'Memories of being in a peaceful, natural setting like a forest or '
            'woodland area',
 'query_10': 'Metacognitive awareness of any gaps in your knowledge about the '
             'specific forest environment you are currently in',
 'query_2': 'Experiences of feeling a sense of tranquility and connection with '
            'nature',
 'query_3': 'Recollections of observing small animals like squirrels in their '
            'natural habitat',
 'query_4': 'Knowledge about the sensory experiences of being in a forest, '
            'such as the sights, sounds, smells, and textures',
 'query_5': 'Memories of times when you felt a deep appreciation for the '
            'beauty and serenity of the natural world',
 'query_6': 'Semantic knowledge about the ecological relationships and '
            'interdependencies within a forest ecosystem',
 'query_7': 'Episodic memories of previous hiking or camping trips in forested '
            'areas',
 'query_8': 'Associations

In [87]:
memory_report_synthesis_generator = LLMResponseGenerator(
    input_tags=["working_memory", "retrieved_memories"],
    output_tag_patterns=["memory_report"],
    prompt_template=memory_report_synthesis_template,
    llm=small_llm,
)

memory_report_response = memory_report_synthesis_generator.generate_response(
    working_memory=test_working_memory,
    retrieved_memories=test_retrieved_memories,
)
memory_report_response
print(memory_report_response["response"])

<memory_report>
As I walk through the serene forest, a sense of peace and tranquility washes over me. The tall, verdant trees and the crisp, fresh air evoke a familiar feeling - a memory from my childhood of a camping trip with my family. I can almost smell the smoky aroma of the campfire as we roasted marshmallows and shared stories under the twinkling stars. That experience of connecting with nature and loved ones left a lasting impression, reminding me of the restorative power of the outdoors.

The current scene also brings to mind a biology lesson I once had, where we delved into the intricate process of photosynthesis and the vital role that trees play in sustaining the delicate balance of our ecosystem. I recall the fascination I felt as I learned how these towering plants convert sunlight, carbon dioxide, and water into the oxygen we breathe. It's remarkable how this fundamental biological process, which I once studied in the classroom, is now unfolding all around me in this ser

In [88]:
working_memory_generator = LLMResponseGenerator(
    input_tags=["working_memory", "memory_report"],
    output_tag_patterns=["updated_working_memory"],
    prompt_template=working_memory_update_template,
    llm=small_llm,
)

working_memory_response = working_memory_generator.generate_response(
    working_memory=test_working_memory,
    memory_report=memory_report_response["memory_report"],
)
working_memory_response
print(working_memory_response["response"])

<updated_working_memory>
The serene forest scene continues to captivate my senses, but now it is infused with a rich tapestry of memories and associations. The tranquil atmosphere evokes a cherished childhood memory of a family camping trip, where the smoky aroma of the campfire and the twinkling stars overhead created a profound sense of connection with nature and loved ones. This recollection is intertwined with a fascination for the intricate process of photosynthesis, a biological lesson that deepened my understanding of the vital role trees play in sustaining our ecosystem.

Overlaying these personal and academic memories is a recent insight from research on the mental health benefits of spending time in nature. The restorative power of immersing oneself in a natural environment, with its ability to reduce stress and enhance well-being, resonates strongly in this moment. I can feel the tension in my mind and body gradually dissipating, replaced by a profound sense of calm and reju

In [89]:
action_decision_generator = LLMResponseGenerator(
    input_tags=["working_memory", "actions"],
    output_tag_patterns=["action_decision"],
    prompt_template=action_decision_template,
    llm=small_llm,
)

actions = (
    "1. Explore the forest to discover new sights and sounds.",
    "2. Sit down and meditate to deepen the sense of peace and tranquility.",
    "3. Take out a notebook and start sketching the trees and wildlife around you.",
)

action_decision_response = action_decision_generator.generate_response(
    working_memory=working_memory_response["updated_working_memory"],
    actions=actions,
)
print(action_decision_response["response"])

<action_decision>
As I continue my peaceful stroll through the serene forest, the rich tapestry of memories and associations woven into this experience captivates my senses and guides my decision-making. The tranquil atmosphere, infused with the cherished recollections of a family camping trip and the fascination for the intricate process of photosynthesis, creates a profound sense of connection with the natural world.

Overlaying these personal and academic memories is the recent insight from research on the mental health benefits of spending time in nature. The restorative power of immersing myself in this natural environment resonates strongly, and I can feel the tension in my mind and body gradually dissipating, replaced by a profound sense of calm and rejuvenation.

Given this context, the action that aligns most closely with my current goals and emotional state is to sit down and meditate to deepen the sense of peace and tranquility. This choice is driven by the desire to fully i

In [16]:
class MemoryDatabase:
    def __init__(self, initial_memories: list[str]):
        self.index = VectorStoreIndex.from_documents(
            [Document(text=memory) for memory in initial_memories],
            embed_model=HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5"),
        )

    def add_memories(self, memories: list[str]):
        for memory in memories:
            self.index.insert(Document(text=memory))
    
    def retrieve(self, query: str, top_k: int = 5):
        retriever = VectorIndexRetriever(index=self.index, similarity_top_k=top_k)
        return retriever.retrieve(query)

@dataclass
class Config:
    small_llm: Anthropic
    large_llm: Anthropic

class Agent:
    def __init__(self):
        self.working_memory = ""
        # TODO: need to represent personality and background. Use a couple different personality test results to give as reference, as well as some core background information

    def process_observations(self, observations):
        """ Process observations and update working memory """
        # small_llm(scratchpad, observations) -> (observation report, relevant memory search queries)
        # retrieve relevant memories
        # small_llm(scratchpad, observation report, memories) -> memory report
        # large_llm(scratchpad, observation report, memory report) -> updated scratchpad
        pass

    # TODO: To make this flexible, take in a list of ActionInterface objects
    # - Each ActionInterface object represents an interface that the agent can interact with, and has a
    # pydantic template for the agent to know the appropriate parameters to pass to the interface and
    # extensive documentation on the interface's purpose and expected behavior
    # - For now, the agent must choose a single interface at a time to interact with
    def choose_action(self):
        """ Choose an action based on working memory """
        # large_llm(scratchpad, action_interfaces) -> choose action interface and parameters
        pass

In [17]:
memories = [
    "I am a human.",
    "I had an apple for breakfast.",
    "My name is John.",
    "My favorite color is blue.",
    "I have a pet cat.",
    "I am 25 years old.",
    "I am a software engineer.",
]

db = MemoryDatabase(memories)

In [26]:
query = "What did I have for breakfast?"
results = db.retrieve(query, top_k=3)
for result in results:
    print(result)

Node ID: 66e16f3a-dd72-40a0-95e0-6100346cc037
Text: I had an apple for breakfast.
Score:  0.799

Node ID: c2fb9d46-be9f-46b3-b89f-69cdb52b1a84
Text: I had a salad for lunch.
Score:  0.652

Node ID: 8ccc5869-346c-4a7f-9153-d213836a07cf
Text: I am 25 years old.
Score:  0.516



In [25]:
db.add_memories(["I had a salad for lunch."])
results = db.retrieve("What did I have for lunch?", top_k=3)
for result in results:
    print(result)

Node ID: c2fb9d46-be9f-46b3-b89f-69cdb52b1a84
Text: I had a salad for lunch.
Score:  0.777

Node ID: 66e16f3a-dd72-40a0-95e0-6100346cc037
Text: I had an apple for breakfast.
Score:  0.689

Node ID: 8ccc5869-346c-4a7f-9153-d213836a07cf
Text: I am 25 years old.
Score:  0.505



In [ ]:
# Use LLM to generate a consistent set of memories, etc to bootstrap an agent. Cache to a text file to save time.
def bootstrap_test_agent():
    pass

# Test the agent on a set of observations. Action space can be a set of possible responses at the level of detail of a choose-your-own-adventure game.
def test_agent():
    pass